# Create an XGBoost model

Using the data created in the previous notebook, we will try to predict the probability of default in the next twelve months. Other situations, such as loan ongoing and loan being repaid will be counted as a non-default

In [1]:
import numpy as np
import pandas as pd

path_to_csv = "../data/mortgage_default_transformed.csv"
df = pd.read_csv(path_to_csv)
df.set_index(df.pop("Unnamed: 0"), inplace=True)

X = df.dropna(axis=0)
y = X.pop("result")

In [8]:
# custom features
X["time_since_start"] = X.time - X.orig_time
X["time_until_end"] = X.mat_time - X.time
X["mortgage_duration"] = X.mat_time - X.orig_time
X["balance_change"] = X.balance_time-X.balance_orig_time
X["LTV_change"] = X.LTV_time - X.LTV_orig_time
X["interest_change"] = X.interest_rate_time - X.Interest_Rate_orig_time
# X["interest_change_as_fraction"] = X.interest_change / X.Interest_Rate_orig_time
X["hpi_change"] = X.hpi_time - X.hpi_orig_time

In [9]:
# select useful columns
used_columns = [
    "time_since_start", "time_until_end", "mortgage_duration",
    "balance_time", "balance_orig_time", "balance_change",
    "LTV_time", "LTV_orig_time", "LTV_change",
    "interest_rate_time", "Interest_Rate_orig_time", "interest_change",
    #"interest_change_as_fraction",
    "hpi_time", "hpi_orig_time",
    "hpi_change", "uer_time"
]

In [10]:
X_data = X.loc[:, used_columns]
X_data.head()

,time_since_start,time_until_end,mortgage_duration,balance_time,balance_orig_time,balance_change,LTV_time,LTV_orig_time,LTV_change,interest_rate_time,Interest_Rate_orig_time,interest_change,hpi_time,hpi_orig_time,hpi_change,uer_time
Unnamed: 0,,,,,,,,,,,,,,,,
0,33,87,120,41061.95,45000.0,-3938.05,24.483867,69.4,-44.916133,9.20,9.2,0.00,225.10,87.03,138.07,4.7
1,51,69,120,30585.43,45000.0,-14414.57,27.202731,69.4,-42.197269,9.20,9.2,0.00,150.91,87.03,63.88,9.2
2,28,93,121,60882.42,63750.0,-2867.58,34.007232,81.8,-47.792768,10.50,10.5,0.00,225.10,97.99,127.11,4.7
3,39,82,121,59023.80,63750.0,-4726.20,48.394751,81.8,-33.405249,9.25,10.5,-1.25,153.35,97.99,55.36,9.0
4,60,61,121,53400.71,63750.0,-10349.29,35.644306,81.8,-46.155694,10.50,10.5,0.00,188.37,97.99,90.38,6.2


In [11]:
# split the data for testing the model
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_data, y,
    train_size=0.7,
    test_size=0.3)

In [12]:
# we will try to select a better model using accuracy scoring
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def score_model(X_train, y_train, X_valid, y_valid, n_estimators, learning_rate=0.1):
    model = XGBClassifier(
                n_estimators = n_estimators,
                learning_rate = learning_rate,
                n_jobs = 5,
                eval_metric = 'mlogloss'
            )
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    
    accuracy = accuracy_score(predictions, y_valid)
    return accuracy

C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [13]:
# shut up warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# we will try different number of estimators to see, which is the best fit
estimator_count = [10, 25, 36, 50, 100]

last_accuracy = 0
for n_estimators in estimator_count:
    accuracy = score_model(
        X_train, y_train, X_valid, y_valid,
        n_estimators=n_estimators,
        learning_rate=0.5 # let's speed things up
    )
    print(f"The accuracy with n_estimators is {n_estimators} with an accuracy of {accuracy}")

The accuracy with n_estimators is 10 with an accuracy of 0.8352805025901024


KeyboardInterrupt: 

In [36]:
some_model = XGBClassifier(
    n_estimators = 100,
    learning_rate = 0.05,
    reg_lambda=0.9,
    reg_alpha=0.1,
    objective='reg:logistic',
    eval_metric = 'rmse',
)
some_model.fit(X_train, y_train)
predictions = some_model.predict(X_valid)
accuracy = accuracy_score(predictions, y_valid)
print(f"Accuracy is {accuracy}")

Accuracy is 0.8358315882288108


This is much better than the previous model, where repayment was a also possible output scenario. By only focusing on the probability of default, we have achieved over 80% accuracy! Let's try calculating the Mean Squared Error next!

In [16]:
import random

arbitrary_predictions = [random.randint(0,1) for _ in range(y_valid.shape[0])]

print(f"Accuracy of randomly generated results is {accuracy_score(arbitrary_predictions, y_valid)}")

Accuracy of randomly generated results is 0.5048495536206327


Just for reference, a randomly generated array of answers would be around 50% accurate at predicting a default. This means, that our model is around 60% better than just randomly guessing. Which is good, but we can make it much better.

In [21]:
hits = 0
total = len(y_valid)
for val1, val2 in zip(predictions, y_valid):
    if val1 == val2:
        hits += 1
print(f"Prediction accuracy is {hits/total}") 

Prediction accuracy is 0.8341232227488151


In [31]:
rand_hits = 0
total = len(y_valid)
for val1, val2 in zip(arbitrary_predictions, y_valid):
    if val1 == val2:
        rand_hits += 1
print(f"Random guessing results in {rand_hits/total} accuracy")

Random guessing results in 0.5048495536206327 accuracy
